In [1]:
#import fwiVis.fwiVis as fv
import s3fs
s3 = s3fs.S3FileSystem(anon=False)
from math import cos, asin, sqrt
import re

import numpy as np
import geopandas as gpd
import pandas as pd
from matplotlib import pyplot as plt
import os
import rioxarray as rio
import xarray as xr
import rasterio
import glob
from shapely.errors import ShapelyDeprecationWarning
from shapely.geometry import Point
import warnings
import folium
import datetime
import time
from folium import plugins
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
import contextily as cx
from shapely.geometry import box
import sys
from datetime import datetime, timedelta

sys.path.insert(0, '/projects/old_shared/fire_weather_vis/base-fwi-vis/')
import fwiVis.fwiVis as fv

In [15]:
def centroid_fire(fireID, year = '2023', path_region = "backup_BOREAL_NRT_3571_DPS_largefire" ):
    fr = fv.load_large_fire(fireID, year = year, path_region= path_region)
    fr = fr[fr.t == fr.t.max()]
    tmp = pd.DataFrame([[fr.lat.iloc[0], fr.lon.iloc[0], fr.farea.iloc[0], path_region]], columns= ["lat", "lon", "farea", "data_source"])
    return(tmp)

In [14]:
 
#fr = fv.load_large_fire('4049', year = '2023', path_region= 'backup_BOREAL_NRT_3571_DPS_largefire') #

In [13]:
#centroid_fire('4049')

['/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F4049_20230430AM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F4049_20230430PM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F4049_20230501AM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F4049_20230501PM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F4049_20230502AM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F4049_20230502PM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F4049_20230503AM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup

/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:584: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lon'] = gdf.centroid.x
/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:585: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lat'] = gdf.centroid.y


,lat,lon,farea
0,53.659499,-115.145972,36.93166


In [86]:
#

from datetime import date
import os

year = None
regnm = 'backup_BOREAL_NRT_3571_DPS_largefire'


def lf_ids(year = None, regnm = 'backup_BOREAL_NRT_3571_DPS_largefire'):
    
    diroutdata = "s3://maap-ops-workspace/shared/gsfc_landslides/FEDSoutput-s3-conus/"

    if year == None:
        year = date.today().year

    if diroutdata.startswith("s3://"):
        # Can't use glob for S3. Use s3.ls instead.
        import s3fs
        s3 = s3fs.S3FileSystem(anon=False)
        s3path = os.path.join(diroutdata, regnm, str(year), "Largefire")
        fnms = [f for f in s3.ls(s3path)]


    fnms.sort()
    ids = []
    for f in fnms:
        fnm_lts = os.path.basename(f) 
        one_id = fnm_lts[1:-11]
        ids.append(one_id)
    tmp_ids = pd.DataFrame(ids, columns=["ids"])
    tmp_ids = tmp_ids.ids.unique()
    return(tmp_ids.values)


In [49]:
s3path

#/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/

's3://maap-ops-workspace/shared/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire'

In [50]:
fnm_lts = os.path.basename(f)

In [51]:
fnm_lts

'F986_20230302AM'

In [58]:
ids = lf_ids()

In [93]:

#fires = pd.DataFrame([[None, None, None, None]], columns= ["lat", "lon", "farea", "data_source"])
fires = []
for n,i in enumerate(ids, start = 0):
    try:
        foo = centroid_fire(i)

    except Exception as e:
        print("Error at ID: ",i,)
        continue

    fires.append([foo.lat.iloc[0], foo.lon.iloc[0], foo.farea.iloc[0], foo.data_source.iloc[0]])
    #print(fires)
    if((n%5 == 0) | (n == (len(ids) -1))):
        print(i)
        fr_pd = pd.DataFrame(fires, columns=["lat", "lon", "farea", "data_source"])
        fr_pd.to_csv("/projects/old_shared/fire_weather_vis/Lightning_analysis/lf_centroids/"+"fire_centroids_" + regnm +".csv")
    
    

[]
Error at ID:  
['/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F1022_20230208PM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F1022_20230209AM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F1022_20230209PM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F1022_20230210AM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F1022_20230210PM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F1022_20230211AM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F1022_20230211PM', '/projects/shared-buckets/gsfc_landslides/FEDSoutp

/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:584: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lon'] = gdf.centroid.x
/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:585: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lat'] = gdf.centroid.y


['/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F1050_20230208AM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F1050_20230208PM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F1050_20230209AM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F1050_20230209PM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F1050_20230210AM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F1050_20230210PM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F1050_20230211AM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup

/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:584: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lon'] = gdf.centroid.x
/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:585: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lat'] = gdf.centroid.y


['/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F10896_20230811AM']


...

/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:584: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lon'] = gdf.centroid.x
/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:585: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lat'] = gdf.centroid.y


['/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F9762_20230811AM']


/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:584: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lon'] = gdf.centroid.x
/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:585: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lat'] = gdf.centroid.y


['/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F986_20230208PM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F986_20230209AM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F986_20230209PM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F986_20230210AM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F986_20230210PM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F986_20230211AM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F986_20230211PM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL

/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:584: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lon'] = gdf.centroid.x
/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:585: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lat'] = gdf.centroid.y


In [92]:
fr_pd = pd.DataFrame(fires)
fr_pd

,0,1,2,3,4
0,lat lon farea ...,lat lon farea ...,lat lon farea ...,lat lon farea ...,lat lon farea ...


In [84]:
pd.DataFrame([fires[1], fires[0]])

ValueError: Must pass 2-d input. shape=(2, 1, 4)